# Simulation of Complex Systems - Chapter 12
**Author**: Artur Gasparyan

In [ ]:
need_upgrade = False
if need_upgrade:
    !pip install --upgrade matplotlib
    !pip install --upgrade networkx

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import math

seed = 69420
np.random.seed(seed)
rng = np.random.default_rng(seed)

# Exercise 12.1

In [ ]:
def generate_er_edges(n_nodes, p_edge):
    edges = np.zeros((n_nodes, n_nodes))
    
    for i in range(n_nodes):
        for j in range(i+1, n_nodes):
            if rng.random() < p_edge:
                edges[[i, j], [j, i]] = 1
                
        
    return edges

In [ ]:
n_nodes_all = [500, 1000, 2000]
p_edge_all = [0.05, 0.01, 0.005]
n_tests = len(n_nodes_all)


for i in range(n_tests):
    n_nodes = n_nodes_all[i]
    p_edge = p_edge_all[i]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
    fig.suptitle(f"Number of edges: {n_nodes}, Edge probability: {p_edge}")
    ax2.set_xlabel(r"degree ($k$)")
    ax2.set_ylabel(r"$p(k)$")

    edges = generate_er_edges(n_nodes, p_edge)
    er_graph = nx.Graph(edges)
    nb_counts = np.sum(edges, axis=0)
    
    counts, bins = np.histogram(nb_counts, density=True)
    n_bins = len(bins)
    last_bin = math.ceil(max(bins))
    
    nb_count_theory = np.zeros(last_bin)
    for k in range(last_bin):
        nb_count_theory[k] = math.comb(n_nodes-1, k) * (p_edge**k) * ((1-p_edge)**(n_nodes-1-k))
        
    gaussian = np.zeros(last_bin)
    count_std = np.std(nb_counts)
    count_mean = np.mean(nb_counts)
    for k in range(last_bin):
        gaussian[k] = math.exp(-0.5*((k-count_mean)/count_std)**2)/(count_std*math.sqrt(2*math.pi))
        
    nx.draw_circular(er_graph, ax=ax1, node_size=50, width=0.02)
    ax2.stairs(counts, bins)
    ax2.plot(np.arange(last_bin), nb_count_theory)
    ax2.plot(np.arange(last_bin), gaussian)
    
    ax2.legend(["Degree", "Theoretical", "Gaussian"])


# Exercise 12.2

In [ ]:
def generate_ws_edges(n_nodes, n_nb, p_rewire):
    assert n_nb % 2 == 0
    
    edges = np.zeros((n_nodes, n_nodes))

    half_range = n_nb // 2
    
    for i in range(n_nodes):
        for j in range(-half_range, half_range+1):
            if j == 0:
                continue
            
            if rng.random() < p_rewire:
                offset = rng.integers(1, n_nodes)
            else:
                offset = j
                
            k = (i+offset) % n_nodes
                
                
            edges[k, i] = 1
            edges[i, k] = 1

                
    # TODO: Rewiring
        
    return edges

In [ ]:
ws_edges = generate_ws_edges(12, 4, 0.2)
ws_graph = nx.Graph(ws_edges)
fig = plt.figure()
nx.draw_circular(ws_graph, with_labels=True)

print(ws_edges)
print(ws_edges==ws_edges.T)
print(ws_edges.sum())